In [0]:
import numpy as np
import tensorflow as tf
from skimage.exposure import rescale_intensity

In [0]:
_integer_types = (np.byte, np.ubyte,          # 8 bits
                  np.short, np.ushort,        # 16 bits
                  np.intc, np.uintc,          # 16 or 32 or 64 bits
                  np.int_, np.uint,           # 32 or 64 bits
                  np.longlong, np.ulonglong)  # 64 bits
_integer_ranges = {t: (np.iinfo(t).min, np.iinfo(t).max)
                   for t in _integer_types}
dtype_range = {np.bool_: (False, True),
               np.bool8: (False, True),
               np.float16: (-1, 1),
               np.float32: (-1, 1),
               np.float64: (-1, 1)}
dtype_range.update(_integer_ranges)
dtype_range

{numpy.bool_: (False, True),
 numpy.float16: (-1, 1),
 numpy.float32: (-1, 1),
 numpy.float64: (-1, 1),
 numpy.int16: (-32768, 32767),
 numpy.int32: (-2147483648, 2147483647),
 numpy.int64: (-9223372036854775808, 9223372036854775807),
 numpy.int64: (-9223372036854775808, 9223372036854775807),
 numpy.int8: (-128, 127),
 numpy.uint16: (0, 65535),
 numpy.uint32: (0, 4294967295),
 numpy.uint64: (0, 18446744073709551615),
 numpy.uint64: (0, 18446744073709551615),
 numpy.uint8: (0, 255)}

In [0]:
#_integer_ranges = {t: (t.min, t.max)
#                   for t in _integer_types}
integer_ranges={
 "<dtype: 'float16'>": (-1, 1),
 "<dtype: 'float32'>": (-3.4028235e+38, 3.4028235e+38),
 "<dtype: 'float64'>": (-1.7976931348623157e+308, 1.7976931348623157e+308),
 "<dtype: 'int16'>": (-32768, 32767),
 "<dtype: 'int32'>": (-2147483648, 2147483647),
 "<dtype: 'int64'>": (-9223372036854775808, 9223372036854775807),
 "<dtype: 'int8'>": (-128, 127),
 "<dtype: 'uint16'>": (0, 65535),
 "<dtype: 'uint32'>": (0, 4294967295),
 "<dtype: 'uint64'>": (0, 18446744073709551615),
 "<dtype: 'uint8'>": (0, 255)}

In [0]:
DTYPE_RANGE = integer_ranges.copy()

In [0]:
if "<dtype: 'float16'>" in DTYPE_RANGE:
  print("yes")

yes


In [0]:
def rescale(image,in_range='image', out_range='dtype'):
  input_image = tf.constant(image)
  sess = tf.InteractiveSession()
  tf.global_variables_initializer().run()
  type(input_image)
  dtype = input_image.dtype
 # imin, imax = intensity_range(image, in_range)
  #imin, imax = intensity_range(image, in_range)
  #omin, omax = intensity_range(image, out_range, clip_negative=(imin >= 0))
  imin, imax = intensity_range(input_image, in_range)
  omin, omax = intensity_range(input_image,out_range,clip_negative=(imin.eval() >= 0))
  print("Calculated omax, omin")
  image=tf.clip_by_value(
    input_image,
    imin,
    imax,
    name=None
)
  print("Clip is done")
  print(tf.math.subtract(imax ,imin))
  image = tf.math.divide(tf.math.subtract(image ,imin), tf.math.subtract(imax ,imin))
  
  return (image * (omax - omin) + omin).eval()
  
  
  

In [0]:
def intensity_range(image, range_values='image', clip_negative=False):
  if range_values == 'dtype':
        range_values = image.dtype
  if str(range_values) == 'image':
        i_min = tf.reduce_min(image)
        i_max = tf.reduce_max(image)
  
  elif str(range_values).strip("") in DTYPE_RANGE:
        i_min, i_max = DTYPE_RANGE[str(range_values).strip("")]
        if clip_negative:
            i_min = 0
  else:
      i_min, i_max = range_values
  return i_min, i_max

In [0]:
image = np.array([51, 102, 153], dtype=np.float16)

In [0]:
rescale(image)

Calculated omax, omin
Clip is done
Tensor("Sub_15:0", shape=(), dtype=float16)


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([0. , 0.5, 1. ], dtype=float16)

In [0]:
rescale_intensity(image)

array([0. , 0.5, 1. ], dtype=float16)

In [0]:
from skimage import data
camera = data.camera()
camera=np.array(camera, dtype=np.float16)

In [0]:
my_function=rescale(camera)
def_functin=rescale_intensity(camera)

Calculated omax, omin
Clip is done
Tensor("Sub_18:0", shape=(), dtype=float16)


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
my_function==def_functin

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [0]:
def rescale_tf(image,in_range='image', out_range='dtype'):
  dtype = input_image.dtype
 # imin, imax = intensity_range(image, in_range)
  #imin, imax = intensity_range(image, in_range)
  #omin, omax = intensity_range(image, out_range, clip_negative=(imin >= 0))
  imin, imax = intensity_range(input_image, in_range)
  omin, omax = intensity_range(input_image,out_range,clip_negative=(imin.eval() >= 0))
  print("Calculated omax, omin")
  image=tf.clip_by_value(
    input_image,
    imin,
    imax,
    name=None
)
  print("Clip is done")
  print(tf.math.subtract(imax ,imin))
  image = tf.math.divide(tf.math.subtract(image ,imin), tf.math.subtract(imax ,imin))
  return (image * (omax - omin) + omin)

In [0]:
from skimage import data
camera = data.camera()
camera=np.array(camera, dtype=np.float16)

In [0]:
input_image = tf.constant(camera)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
out_put=rescale_tf(input_image)

Calculated omax, omin
Clip is done
Tensor("Sub_33:0", shape=(), dtype=float16)


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
output_tf=out_put.eval()
output_np=rescale_intensity(camera)

In [0]:
np.array_equal(output_tf,output_np)

True